In [ ]:
#pip install git+https://github.com/dreamquark-ai/tabnet

In [ ]:
#!pip install pytorch-tabnet

In [ ]:
import pandas as pd
import datetime
from datetime import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('../dataset/dataset.csv', encoding = 'shift_jis')
df

In [ ]:
product_code = list(range(1,10+1))
store_code = [617,651,833,921,922,934,937,1190,1250,1261,1236,671,672]

dataset = [[None]*len(store_code) for _ in range(len(product_code))]

for i in range(len(store_code)):
    store_df = df[df['ShopC'] == store_code[i]]
    for j in range(len(product_code)):
            dataset[j][i] = store_df[store_df['ProductC'] == product_code[j]]

TEST_DAYS = 30
drop_list = ['SalesDate','ShopC','ProductC','SalesCate',
            'Year','PreAmo','WindAve','VapAve','CloAve','0Check',]


train = []
test = []

for i in range(len(store_code)):
    for j in range(len(product_code)):
        train.append((dataset[j][i])[0:-TEST_DAYS].drop(drop_list, axis=1))
        test.append((dataset[j][i])[-TEST_DAYS:].drop(drop_list, axis=1))

train_data = pd.DataFrame(columns = df.columns.drop(drop_list).values)

for i in train:
    train_data = pd.concat([train_data, i], axis = 0)


from sklearn.model_selection import train_test_split
random_state = 0
X_train, X_valid, y_train, y_valid = train_test_split(train_data.iloc[:, :-1].values, train_data.iloc[:, [-1]].values, test_size=.3, random_state=random_state)

X_train = X_train.astype(np.float64)
X_valid = X_valid.astype(np.float64)

y_train = y_train.astype(np.int64)
y_valid = y_valid.astype(np.int64)

# print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape)


In [ ]:
from pytorch_tabnet.pretraining import TabNetPretrainer
from sklearn.metrics import mean_squared_error
import torch

SEED = 53

unsupervised_model = TabNetPretrainer(n_d=8, n_a=8, n_steps=5, gamma=1.6,
                        n_independent=2, n_shared=2,
                        seed=SEED, lambda_sparse=1e-3,
                        optimizer_fn=torch.optim.Adam,
                        optimizer_params=dict(lr=2e-2),
                        mask_type="entmax",
                        scheduler_params=dict(mode="min",
                                                patience=5,
                                                min_lr=1e-5,
                                                factor=0.9,),
                        scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                        verbose=5
                        )

unsupervised_model.fit(
    X_train,
    eval_set=[X_valid],
    max_epochs=100 , patience=30,
    batch_size=256, virtual_batch_size=128,
    num_workers=0,
    drop_last=False,
    # pretraining_ratio=0.8,
)

In [ ]:
import torch
from pytorch_tabnet.tab_model import TabNetRegressor

model = TabNetRegressor(n_d=8, n_a=8, n_steps=5, gamma=1.6,
                        n_independent=2, n_shared=2,
                        seed=SEED, lambda_sparse=1e-3,
                        optimizer_fn=torch.optim.Adam,
                        optimizer_params=dict(lr=2e-2),
                        mask_type="entmax",
                        scheduler_params=dict(mode="min",
                                                patience=5,
                                                min_lr=1e-5,
                                                factor=0.9,),
                        scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                        verbose=5
                        )

model.fit(
  X_train, y_train,
  eval_set=[(X_valid, y_valid)],
  max_epochs=100,
  patience=30,
  eval_metric={'rmse', 'mae'}, # 'rmsle', 'mse', 'mae'
  from_unsupervised=unsupervised_model
)



In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math

rmse_list = []
mae_list = []
error_list = []
max_error_list = []

count = 0
for data in test:
    if len(data) == 30:
        test_data = pd.DataFrame(data)
        X_test = test_data.iloc[:, :-1].values.astype(np.float64)
        y_test = test_data.iloc[:, [-1]].values.astype(np.int64)
        rmse_score = math.sqrt(mean_squared_error(model.predict(X_test), y_test))
        mae_score = mean_absolute_error(model.predict(X_test), y_test)
        rmse_list.append(rmse_score)
        mae_list.append(mae_score)
        error_list.append(float(mae_score/(sum(y_test)/len(data))*100))
        max_error = 0
        for index, i in enumerate(model.predict(X_test)):
            max_error = max(max_error, abs(i - y_test[index]) / y_test[index] * 100)
        max_error_list.append(float(max_error))
        print(store_code[count//10], "-", product_code[count%10])
        plt.figure(figsize=(10, 6), dpi=100)
        plt.plot(model.predict(X_test),c='r')
        plt.plot(y_test, c='b')
        plt.show()

        # print(rmse_score,mae_score)
    else:
        rmse_list.append("None")
        mae_list.append("None")
        error_list.append("None")
        max_error_list.append("None")
    count += 1
# print(f"FINAL TEST SCORE: {test_score}")
# print(f"FINAL TEST SCORE: {test_score}")


In [ ]:

result_df = pd.DataFrame(columns=['ShopC', 'ProductC','RMSE','MAE','ERROR', 'MAX'])
count = 0
for store in range(len(store_code)):
    for product in range(len(product_code)):
        result_df = result_df.append({'ShopC': int(store_code[store]), 'ProductC': int(product_code[product]), 'RMSE': rmse_list[count] , 'MAE': mae_list[count], 'ERROR' : error_list[count], 'MAX' : max_error_list[count]}, ignore_index=True)
        count += 1
result_df.to_csv('../dataset/result.csv', encoding = 'shift_jis',index=None)